import pandas as pd

In [1]:
import pandas as pd

In [2]:
tripadv_review = pd.read_csv(r'C:\Users\Dell\Desktop\Attractions\The Mall of Cyprus.csv', error_bad_lines=False);

In [3]:
import gensim
from gensim.utils import simple_preprocess
from gensim.parsing.preprocessing import STOPWORDS
from nltk.stem import WordNetLemmatizer, SnowballStemmer
from nltk.stem.porter import *
import numpy as np
np.random.seed(2018)

In [4]:
import nltk
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Dell\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [5]:
def preprocess(text):
    result = []
    for token in gensim.utils.simple_preprocess(text):
        if token not in gensim.parsing.preprocessing.STOPWORDS and len(token) > 3:
            result.append(token)
    return result

In [6]:
processed_reviews = tripadv_review['r_text'].map(preprocess)
processed_reviews

0      [excellent, place, going, parking, place, plen...
1      [typical, modern, european, shopping, center, ...
2      [stopped, mall, cyprus, ikea, nicosia, mall, s...
3      [renovation, mall, year, area, restaurant, caf...
4      [favorite, mall, nicosia, good, variety, shops...
                             ...                        
101    [mall, located, city, center, reach, course, p...
102    [visiting, mall, cyprus, tour, guide, need, st...
103    [mall, cyprus, longer, best, mall, shop, kings...
104    [winter, rain, cold, summer, high, temperature...
105    [happen, nicosia, weather, enjoy, outdoors, he...
Name: r_text, Length: 106, dtype: object

In [7]:
import gensim.corpora as corpora
# Create Dictionary
id2word = corpora.Dictionary(processed_reviews)

# Create Corpus
texts = processed_reviews

# Term Document Frequency
corpus = [id2word.doc2bow(text) for text in texts]

# View
print(corpus[:1])

[[(0, 1), (1, 1), (2, 1), (3, 1), (4, 2), (5, 1), (6, 1), (7, 1), (8, 1)]]


In [8]:
lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                           id2word=id2word,
                                           num_topics=7, 
                                           random_state=50,
                                           update_every=1,
                                           chunksize=120,
                                           passes=5,
                                           alpha='auto',
                                           eta='auto',
                                           per_word_topics=True,
                                           iterations = 25,
                                           eval_every = None)

In [9]:
from pprint import pprint

In [10]:
pprint(lda_model.print_topics())
doc_lda = lda_model[corpus]

[(0,
  '0.048*"mall" + 0.023*"good" + 0.020*"shops" + 0.015*"nicosia" + '
  '0.014*"food" + 0.012*"cyprus" + 0.011*"fast" + 0.010*"shopping" + '
  '0.009*"variety" + 0.009*"coffee"'),
 (1,
  '0.028*"mall" + 0.023*"shops" + 0.012*"special" + 0.011*"shop" + '
  '0.011*"zara" + 0.011*"small" + 0.011*"europe" + 0.010*"nicosia" + '
  '0.009*"parking" + 0.008*"public"'),
 (2,
  '0.035*"shops" + 0.030*"good" + 0.029*"mall" + 0.025*"food" + 0.023*"place" '
  '+ 0.020*"parking" + 0.019*"cinema" + 0.016*"shopping" + 0.015*"cyprus" + '
  '0.013*"available"'),
 (3,
  '0.055*"mall" + 0.021*"shops" + 0.018*"nicosia" + 0.014*"restaurants" + '
  '0.013*"small" + 0.013*"cyprus" + 0.012*"good" + 0.011*"shopping" + '
  '0.010*"park" + 0.009*"shop"'),
 (4,
  '0.017*"shopping" + 0.015*"place" + 0.015*"birds" + 0.012*"food" + '
  '0.012*"free" + 0.012*"lots" + 0.012*"mall" + 0.011*"absolutely" + '
  '0.011*"shops" + 0.010*"parking"'),
 (5,
  '0.017*"shopping" + 0.016*"place" + 0.015*"parking" + 0.013*"good"

In [11]:
import pyLDAvis.gensim
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim.prepare(lda_model, corpus, dictionary=lda_model.id2word)
vis

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
3      0.100219  0.049066       1        1  27.596161
2     -0.052774  0.071401       2        1  23.446901
6     -0.084732  0.029351       3        1  16.208920
1      0.069055 -0.036046       4        1   9.652781
0      0.015398  0.044521       5        1   9.130179
5     -0.039396 -0.060439       6        1   8.026075
4     -0.007770 -0.097854       7        1   5.938983, topic_info=          Term       Freq      Total Category  logprob  loglift
4        place  24.000000  24.000000  Default  30.0000  30.0000
29        mall  90.000000  90.000000  Default  29.0000  29.0000
23        good  40.000000  40.000000  Default  28.0000  28.0000
3      parking  28.000000  28.000000  Default  27.0000  27.0000
16    shopping  34.000000  34.000000  Default  26.0000  26.0000
..         ...        ...        ...      ...      ...      ...
33       small   0.629044  18.482330   Topic7  -5.4339  -0.5567
166  starbucks   0.619115   7.149682   Topic7  -5.4498   0.3771
141      clean   0.619646   9.348381   Topic7  -5.4490   0.1098
21      cyprus   0.620773  25.914888   Topic7  -5.4472  -0.9080
235       want   0.614917   6.579988   Topic7  -5.4566   0.4533

[456 rows x 6 columns], token_table=      Topic      Freq        Term
term                             
201       4  0.339431  absolutely
201       7  0.678861  absolutely
103       3  0.686015    achieved
583       6  0.408494  activities
583       7  0.408494  activities
...     ...       ...         ...
265       7  0.184104       worth
318       1  0.367515        zara
318       4  0.367515        zara
318       5  0.122505        zara
318       7  0.122505        zara

[620 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[4, 3, 7, 2, 1, 6, 5])